In [1]:
using DifferentialEquations, ModelingToolkit, Distributions, Plots, LinearAlgebra


(process:21736): GLib-GIO-WARNING **: 12:09:13.333: Unexpectedly, UWP app `Evernote.Evernote_10.36.4.0_x86__q4d96b2w5wcc2' (AUMId `Evernote.Evernote_q4d96b2w5wcc2!Evernote') supports 1 extensions but has no verbs


In [4]:
using Symbolics: scalarize

In [5]:
@variables t
D = Differential(t)

(::Differential) (generic function with 2 methods)

In [15]:
function Mass(; name, m = 1.0, xy = [0.,0.], u = [0., 0.])
    ps = @parameters m=m
    sts = @variables pos[1:2](t) = xy v[1:2](t) = u
    eqs = scalarize(D.(pos) .~ v)
    ODESystem(eqs, t, [pos..., v...], ps; name)
end


Mass (generic function with 1 method)

In [7]:
function Spring(; name, k = 1e4, l = 1.)
    ps = @parameters k=k l=l
    @variables x(t) dir[1:2](t)
    ODESystem(Equation[], t, [x, dir...], ps; name)
end

Spring (generic function with 1 method)

In [10]:
function connect_spring(spring, a, b)
    [
        spring.x ~ norm(scalarize(a .- b))
        scalarize(spring.dir .~ scalarize(a .- b))
    ]
end
    

connect_spring (generic function with 1 method)

In [11]:
spring_force(spring) = -spring.k .* scalarize(spring.dir) .* (spring.x - spring.l) ./ spring.x

spring_force (generic function with 1 method)

In [13]:
m = 1.0
xy = [1., -1.]
k = 1e4
l = 1.
center = [0., 0.]
g = [0., -9.81]

2-element Vector{Float64}:
  0.0
 -9.81

In [16]:
@named mass = Mass(m=m, xy=xy)
@named spring = Spring(k=k, l=l)

Model spring with 0 equations
States (3):
  x(t)
  dir[1](t)
  dir[2](t)
Parameters (2):
  k [defaults to 10000.0]
  l [defaults to 1.0]

In [17]:
eqs = [
    connect_spring(spring, mass.pos, center)
    scalarize(D.(mass.v) .~ spring_force(spring) / mass.m .+ g)
    ]

5-element Vector{Equation}:
 spring₊x(t) ~ sqrt(abs2(mass₊pos[1](t)) + abs2(mass₊pos[2](t)))
 spring₊dir[1](t) ~ mass₊pos[1](t)
 spring₊dir[2](t) ~ mass₊pos[2](t)
 Differential(t)(mass₊v[1](t)) ~ (-spring₊k*(spring₊x(t) - spring₊l)*spring₊dir[1](t)) / (mass₊m*spring₊x(t))
 Differential(t)(mass₊v[2](t)) ~ (-spring₊k*(spring₊x(t) - spring₊l)*spring₊dir[2](t)) / (mass₊m*spring₊x(t)) - 9.81

In [22]:
@named _model = ODESystem(eqs, t)
@named model = compose(_model, mass, spring)
sys = structural_simplify(model)

LoadError: UndefVarError: p not defined

In [20]:
prob = ODEProblem(sys, [], (0., 3.))


ODEProblem with uType Vector{Term{Float64, Nothing}} and tType Float64. In-place: true
timespan: (0.0, 3.0)
u0: 4-element Vector{Term{Float64, Nothing}}:
 [0.0, 0.0][i]
 [0.0, 0.0][i]
 [1.0, -1.0][i]
 [1.0, -1.0][i]

In [21]:
sol = solve(prob, Rosenbrock23())


LoadError: MethodError: no method matching oneunit(::Type{Any})
[0mClosest candidates are:
[0m  oneunit(::Type{Union{Missing, T}}) where T at C:\Users\micho\AppData\Local\Programs\Julia-1.7.2\share\julia\base\missing.jl:105
[0m  oneunit(::Type{T}) where T at C:\Users\micho\AppData\Local\Programs\Julia-1.7.2\share\julia\base\number.jl:358
[0m  oneunit(::T) where T at C:\Users\micho\AppData\Local\Programs\Julia-1.7.2\share\julia\base\number.jl:357
[0m  ...

In [ ]:
plot(sol)